<a href="https://colab.research.google.com/github/EdRey05/Resources_for_Mulligan_Lab/blob/main/Tools%20for%20students/Eduardo%20Reyes/06_METABRIC_KM_Plot_Synthetic_Letality_Batch_%5BColab%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Notebook to generate Kaplan-Meier survival probability plots based on gene expression***

**Original data from:** *METABRIC*

**Publication DOI:** *10.1038/ncomms11479*

**Data downloaded from:** *https://www.cbioportal.org/study/summary?id=brca_metabric*

**Notebook made by:** *Eduardo Reyes-Alvarez (Ph.D. candidate)*

**Affiliation:** *Dr. Lois Mulligan's lab, Queen's University.*

**Contact:** *eduardo_reyes09@hotmail.com*

**Date of latest version:** December 20, 2020.

## 0-Introduction

**Explanation of what this notebook does**

This notebook generates Kaplan-Meier survival estimate curves from data obtained from a breast cancer study (METABRIC) corresponding to this publication: https://pubmed.ncbi.nlm.nih.gov/27161491/

From that data file, we used the mRNA Seq expression levels and clinical information of patients. We focused this study on ER+ patients within the dataset and looked for expression of RET and other genes that we obtained as potential candidates for novel interactions with RET, through a synthetic lethality study.

This notebook corresponds to a second batch of genes screened to evaluate any possible relationship between the expression levels and overall survival of patients. (names of genes can be seen in the variable **input2**). In order to run this notebook, you need to mount a drive (google account), which should contain the input files required and the resulting graphs are saved into the drive folder specified.

***Note:*** This notebook contains some dataset-unique steps, so it is not possible to use it for other studies without some modifications.

**Instructions**

-Create a Google account and log in

-In this notebook, click File -> Save a copy in Drive

-Go to Google Drive

-You should have a "Colab Notebook" folder, open it

-You should see this notebook there

-Upload there the input files with clinical and RNA Seq data (organize in subfolders if you want)

-Synchronize your Google Drive account (run the following code box, click the link that will appear, copy the code by clicking in the symbol on the right side of it, paste it in the blank that will appear and click enter)

-Proceed with the next code box

In [1]:
#This is needed when working in Google Colab to synchronize google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#Double click this box, edit the following 5 things and read the following lines

#01-Clinical data file directory (in your Drive)
clinical_dataset_directory = "/content/drive/MyDrive/Colab Notebooks/Mulligan Lab - PhD/KM_plots/METABRIC_2012_2016/input_files/2-clinical_23Q2.txt"

#02-RNA Seq data file directory (in your Drive)
RNA_dataset_directory = "/content/drive/MyDrive/Colab Notebooks/Mulligan Lab - PhD/KM_plots/METABRIC_2012_2016/input_files/2-RNA_23Q2.txt"

#03-File name for excel output and directory for graphs (in your Drive)
output_directory = "/content/drive/MyDrive/Colab Notebooks/Mulligan Lab - PhD/KM_plots/METABRIC_2012_2016/output_files/SyntheticLetality_BATCH"
output_file_name = "My_File_01.xlsx"

#04-Write the gene of reference
input1 = "RET"

#05-Write the genes of interest, follow the example:
input2 = [
    "GPR25", "GALR2", "TNFRSF1B", "HBA1", "CDK18", "KLF13", "BCKDK", "NPBWR1", "ELL", "EEPD1",
    "APOE", "HOXD11", "SOX1", "SCRT1", "ZNF497", "CA10", "LRRC15", "RFXANK", "PPP4C",
    "STK32A", "POU3F3", "MARS", "PHKG2", "DPEP2", "PHGDH", "ETNK2", "ATP5L", "AKT1", "ADARB1",
    "SGK3", "ZNF680", "SRSF2", "PSMB7", "PITPNM1", "GDF7", "CEBPD", "ROR2", "ASCL1", "PA2G4",
    "FER", "CITED2", "B3GNTL1", "FJX1", "ANKRD46", "TUBA8", "USP4", "P2RX2", "GSR", "NDUFA8",
    "FFAR1", "PLEKHO1", "PEX10", "H3F3B", "C1orf151", "RS1", "PRKCZ", "MAPK4", "B3GNT6", "ANKK1",
    "STK38", "GPER", "ZNF205", "MAP4K5", "FGFR1", "FZD2", "PIK3C2A", "POU4F3", "RIMBP2", "CELF5",
    "RAC1", "ATM", "MAPK11", "FAM19A3", "DOM3Z", "KRT6C", "CHEK2", "ITPKA", "DCTN3", "LRCH1",
    "MNX1", "KCNJ14", "FBXO17", "RGS17", "EPHB6", "HK3", "SMG6", "ZFYVE27", "CEBPE", "LHX6",
    "ELF5", "VRK3", "LCMT2", "OR10H1", "GRK7", "OR10J1", "TBCA", "PTK2", "DPYSL5",
    "PIK3CD", "RPUSD4", "SYT7", "INS", "CST6", "SP6", "RELT", "ZNF366", "MCCD1", "LPXN",
    "C10orf72", "NSUN5C", "ASB2", "ZNF672", "OCLN", "ASB16", "C19orf39", "KCNK12", "PRICKLE4", "SLC10A5",
    "PLEKHO2", "EID2", "C1orf122", "SRY", "SPATA4", "MAK16", "CORO6", "CRISP3", "ASB14",
    "GAL", "FBXO27", "ATAD3A", "FREM2", "CPXM1", "RNLS", "GPR78", "SCD", "FOXG1", "RICTOR",
    "EPHB2", "MMACHC", "EXD3", "RFX1", "HBD", "CDK16", "EPX", "NOMO3", "PAK6", "NKD2",
    "CSNK2B", "SEMA4B", "FOXJ1", "IRX1", "USPL1", "CKM", "AGTPBP1", "PNMT", "PTPRM", "TSPYL5",
    "CXCL1", "C3orf25", "CYB5D2", "IPO5", "PPAP2C", "NCAM1", "HBZ", "FBXW5", "MBD3", "SH2D2A",
    "PFKFB3", "TRPM6", "NTHL1", "ADAM19", "PRKACA", "CYP4F22", "ZNF74", "FZD3", "SPEG", "FAM123C",
    "VWF", "SSX6", "PTPRCAP", "GPD1", "PSMA3", "RUVBL1", "CD163", "LBH", "HAUS6",
    "FAM184A", "SRMS", "AURKA", "ATP5G2", "BMPR2", "RPS6KA2", "ATMIN", "ITFG2", "FAM19A5", "CHMP2B",
    "GPR97", "FGFR3", "CDK4", "MRPS31", "NAP1L2", "TMC4", "NPAS4", "FAM8A1", "MAPK14", "RIOK2", "HNRNPK", "PQBP1",
    "PARN", "DNAH3", "MN1", "BCL7A", "CARD9", "PRKCD", "CNP", "MKNK2", "PODN", "GRPEL1",
    "ATP6V0E1", "GFI1B", "SH3GL1", "BHMT2", "ARNTL", "BACE2", "TGFB1", "SLC9A3R2", "PDCD2", "ACVR1",
    "ATP1B2", "SFTPD", "LRRC4C", "CELF3", "TAX1BP3", "GJA3", "XRCC5", "LILRB5", "RWDD3", "USE1",
    "TRIB3", "PET112L", "TMEM177", "UBQLN2", "SIRT2", "NDUFS7", "TNK2", "STK39", "IGFL1", "JUN",
    "BTK", "ODAM", "KCTD11", "A1BG", "TIE1", "MYO7A", "CFHR3", "CCDC22", "RAB25", "SETD4",
    "BEST1", "TMEM2", "LMO4", "CHCHD6", "TIMM13", "FIGNL1", "STRADA", "GAS7", "PREB", "CALM1",
    "LMTK3", "FASTKD3", "AEBP1", "FN3KRP", "CTRL", "DUX2", "C13orf15", "HOXD1", "CST5", "PLIN5",
    "NCBP2", "MTSS1", "MARCKS", "PDPK1", "KRAS", "PTMS", "HTRA3", "PLTP", "KLK12", "TCF7L1",
    "ATP1B3", "IFT140", "HIST1H2BD", "MAPK8", "TNK1", "KRTAP3-2", "C3orf21", "C12orf35", "PEX3", "FCGR2A",
    "ZNRF2", "GCK", "MRPL32", "TTPAL", "CACNG3", "CRYGB", "MAP3K9", "PPP2R2D", "HOXC9", "DYRK3", "RCE1",
    "PANK4", "NIPSNAP1", "SYNGR1", "TAS2R14", "SLC4A4", "FEZF1", "DDR2", "IL18", "C16orf78", "MLXIP",
    "SLITRK3", "ZNF317", "ONECUT2", "ZNF273", "ZBTB41", "AKTIP", "SCAP", "TCEB3B", "ZG16B", "SMOC2",
    "DPP4", "ZNF705A", "DGKG", "CYP4V2", "SNX3", "LYSMD4", "CXorf58", "DLST", "SDHAF2", "PHF23",
    "S1PR2", "C21orf37", "LTBP4", "C7orf16", "SLC41A1", "PTN", "PTPRN2", "CAPN1", "CCR9", "PLD1",
    "C14orf48", "GPX7", "DNAJC21", "ENTPD7", "MET", "ZXDC", "LMNA", "PLEKHJ1", "FGF21", "ASB9",
    "FRG2", "PDE6G", "GALNT3", "AGRP", "FHL1", "DUSP5", "KRT1", "UFSP2", "ACSF2", "ANKRD12",
    "NCAPG2", "IGFBP1", "KLHDC8A", "OMA1", "CEPT1", "CDK6", "MDS1", "CTSD", "GOLGB1", "ARL5A",
    "GUCY2D", "NR1D1", "BANK1", "MCM3", "STK4", "DIS3L2", "CACNA1F", "FAM49A", "MRPS2", "GPR12",
    "PSKH2", "ZMYM6", "DNAJC9", "PTPN23", "CCDC7", "PPP2R1A", "EHD2", "ACACB", "MAP3K8", "GCC1",
    "TRIOBP", "PRLH", "ZBTB22", "IP6K1", "GUF1", "TLX3", "MAP2K2", "PURA", "CDKN2B", "HRK", "UTS2R",
    "NHLH2", "GRK1", "PTGER1", "MESP2", "ZNF85", "LIAS", "ZNF689", "SDHB", "ZNRF4", "CCDC71",
    "FAU", "ATF5", "POLD4", "TGFB1I1", "HOXC12", "MBNL3", "MAP2", "CDKN2A", "OAF", "RUNX3",
    "ZNF142", "CDK10", "NEUROG3", "PDGFRL", "GPSM3", "TMEM158", "DFFB", "CKB", "CD300C",
    "CPS1", "BRI3", "SSTR4", "CSNK1E", "BOLA1", "SIAH1", "DMBX1", "MSRB2", "PLEKHF1", "FOXP3",
    "HBA2", "DIRC1", "TBX2", "SUV420H1", "KCNN3", "FOXB2", "TCTA", "CUTA", "CNOT3", "GRM4",
    "JUND", "TLX2", "ACTL8", "C14orf149", "COMMD5", "ZMYND15", "UBE2S", "TYROBP", "ADAMTS7",
    "CSNK1G2", "NAA10", "KLF2", "HLA-C", "CRABP1", "ARX", "IQCB1", "C11orf24", "UBR5", "LIMS1",
    "SEMA3A", "LOXL1", "ROBO4", "TWIST1", "PIK3AP1", "ASCL3", "PIWIL4", "TCAP", "B3GALT6", "MAP3K15",
    "HSPB1", "NOX3", "CHST5", "EGFL8", "TNFRSF13C", "S100A12", "ASCL4", "PPP2R3C", "C4A", "NKX2-8",
    "BEX1", "RPS16", "FAF2", "SEPT4", "ETV5", "FYN", "CDH13", "TNFSF12", "PRMT2", "LGI4", "FEM1A",
    "SUN5", "ZFP82", "RNF166", "NCS1", "ZFP36L2", "FXYD6", "C9orf96", "HOXB6", "FGF22", "CS",
    "NVL", "NRXN2", "AATK", "CHST13", "RNF44", "UNCX", "RAI1", "IDH3G", "MAP7", "UMPS",
    "VPS4B", "SEC23IP", "VN1R1", "NARG2", "TRPC4", "FAAH", "HM13", "SLFN5", "C5orf56", "LYNX1",
    "BBC3", "SOD3", "BCL2L10", "MRPL30", "ANKRD17", "OVOL1", "OR6K6", "RNF20", "MYLK4", "HNRNPCL1",
    "SCN11A", "FCRL6", "TSSK6", "FADS2", "PGR", "PAX2", "ADCK4", "MPG", "FZD5", "HIC1",
    "CCNI", "ZNF696", "POU4F1", "SEPSECS", "CDKN2AIP", "TIGD4", "MAGEB1", "MRPS12", "PHEX",
    "SLAMF7", "MYH10", "TFAP2A", "AQP5", "PCGF3", "ACTA2", "WDR47", "MTMR8", "RPS15", "HEATR5B",
    "NRAP", "CXCL9", "NMNAT2", "KRTAP13-2", "RBCK1", "AURKAIP1", "QTRT1", "SH3BP4", "NFIL3", "AMELX",
    "GTF3C4", "ADIG", "SLC6A18", "ZSCAN18", "PRKG2", "CLP1", "CEP72", "UBE2A", "MDFI", "SCO2",
    "XCL1", "SRGAP1", "APC2", "COX5B", "HYAL3", "ALOX5", "TAS1R2", "GLB1L3", "MYCBPAP", "DNAJC4", "FAM72A",
    "GTF3C5", "DCAF11", "KIR3DL2", "HMGN1", "POTEE", "ADAM18", "ZNF175", "OGN", "GPR15", "TBRG4",
    "XRCC6", "APOLD1", "ZNF575", "SEMA7A", "ETV2", "PRMT6", "SEPT12", "LMCD1", "DNAJC5G", "LEP",
    "MBP", "ZNF187", "MATK", "FBXL6", "DUX4", "WT1", "TBC1D9", "GPRIN3", "FOXB1", "IL3",
    "ZNF620", "TBXAS1", "NRGN", "PPP1CA", "NMT2", "HIST1H1T", "IRAK1", "ZKSCAN1", "WDR17",
    "TGFB2", "SOD1", "KLF15", "LRRC8A", "RPA1", "MTAP", "EPHA5", "ZNF785", "HMX1", "RPS21",
    "SCYL2", "FOXM1", "CHRM2", "DAK", "PAGE4", "CSK", "PCDHA7", "RAPGEFL1", "MYH7", "NOS3",
    "CFLAR", "PHKB", "CCND3", "APOL5", "PDZRN3", "SPTB", "C22orf36", "ARL13B", "AOX1", "TCEAL6",
    "GSPT1", "CAMKMT", "MFSD2", "MUTED", "RWDD4A", "LCN1", "FANCG", "CCDC109B", "TTBK1", "SLC18A3",
    "KAT2A", "CXXC1", "FSTL1", "MYL2", "KLF12", "TGIF2LX", "ABHD16B", "LSM8", "PF4", "SALL4",
    "SLC10A3", "MAGEC2", "C6orf125", "TACR2", "COMMD3", "MCAM", "FOXD3", "PCYOX1", "GRIK1", "ZNF280B",
    "RAP1GDS1", "GNB2", "NUDCD1", "DUX3", "MAPKAPK3", "NUAK2", "GPR31", "TOPBP1", "DHX16", "PLAA",
    "PI4K2A", "CCDC82", "KDSR", "EIF3F", "TK1", "PRDM13", "RAB9A", "MICALCL", "DLG3", "ZFP37",
    "DBNL", "ING3", "CXXC5", "ABLIM2", "ZBTB25", "ZNF177", "SLC4A10", "SRGAP2", "GATA5", "ZNF496",
    "CHMP6", "LTA", "FNIP1", "OVOL2", "TIMELESS", "CNOT10", "PCBD2", "CYC1", "ALPL",
    "ATP13A1", "VDR", "PPP1R2P9", "NEUROG2", "AGBL3", "RAX", "SCNN1B", "USP31", "TMEM144", "FAIM",
    "DNAJB12", "COL5A1", "SP100", "KDM1B", "PADI3", "SERPINA7", "RBX1", "NDUFA5", "SDR42E1", "C1orf9",
    "TSKU", "ALOX12B", "JARID2", "BMP1", "GATA6", "GCG", "MORF4", "CDC42EP4", "GPR157", "RAB33A",
    "S1PR3", "FBXO30", "DUSP1", "IL3RA", "NEK2", "CALCR", "PGK2", "PTBP2", "OR2T10", "RPL37A",
    "SULT2B1", "POR", "MRPS23", "TNFRSF6B", "TPM2", "KLK2", "DAZL", "PRR3", "ZIM3", "LCT",
    "DUOXA2", "SCD5", "RPL27", "BATF2", "ALPK1", "COL9A3", "DIAPH3", "UROD", "TESC", "NDUFA13", "PPFIA4",
    "WDR6", "PAGE3", "PLAC1", "RHOD", "ACAD8", "EFCAB6", "HS3ST3A1", "RBKS", "MMP2", "TTC31",
    "ETV1", "FBN2", "HRH3", "POGK", "CD2AP", "SFTPC", "SPSB1", "IRX2", "ETV3", "CD1A",
    "SMC4", "ZNF805", "CLIC3", "HYAL1", "ARHGAP10", "PLVAP", "ZFAND1", "ZC3H3", "OR7E5P", "TMEM140",
    "OSGIN2", "GNAZ", "AWAT1", "NAA30", "SELPLG", "KIAA0090", "EEF1B2", "RPS17P12", "UBN1", "HJURP",
    "MRPL43", "JPH4", "HOXA10", "CTNNBIP1", "PAX6", "STXBP3", "IRAK2", "XAGE2", "CCDC93", "UBA52",
    "ZBTB44", "UBL4A", "IKBKE", "ZSCAN10", "PPAT", "KAZALD1", "SAP30BP", "KCNH1", "SNRPA",
    "CCDC18", "TCEAL4", "CFDP1", "NACAD", "HMHA1", "CDR2L", "ADRA2B", "XPC", "SHPK", "RHBDD1",
    "IL1F7", "SUMO4", "NTAN1", "NLGN4Y", "EGLN1", "DUSP23", "C13orf16", "NDUFAF1", "UBA6", "RAB5A",
    "CAMK2A", "COPZ1", "PDSS2", "AKAP3", "PSMB5", "NOTCH4", "GLYCTK", "LASS6", "CYP39A1",
    "PRPSAP1", "ROR1", "C21orf59", "TMBIM1", "SYTL1", "CDH19", "PSMB1", "MUT", "MSGN1", "MARK1", "C6orf150",
    "CDC25A", "C17orf28", "CPAMD8", "FBXO2", "PI4KA", "COL24A1", "NRBP2", "HMGB3", "CYB561", "DNHD1",
    "HSD11B1L", "NMUR1", "SSTR3", "CCT8L2", "ATP8A1", "PDCD4", "MLLT11", "C13orf1", "MRPS18A", "CSRP1",
    "ALS2CL", "LYRM4", "TRIM13", "RAB20", "TAS2R9", "TREML1", "PDCL", "SLC4A2", "DMRT1", "TGIF1",
    "RNF167", "ING5", "E2F4", "SLCO1B3", "HCFC2", "ARL8B", "ITIH1", "TMEM5", "DDX28", "FSD1L",
    "UACA", "EFHC1", "FASTK", "FZD9", "GAMT", "BATF", "FAR1", "NDUFAF2", "ZNF26", "COLEC11",
    "AQP12A", "MCART2", "WNT2", "PCDH1", "DUSP26", "SCGB1A1", "CRYBA4", "KRTAP19-3", "USP39", "DNMT3A",
    "MPO", "COMT", "SRGAP3", "APAF1", "RNF122", "CCNJ", "ENG", "KCNH6", "STT3A", "FAM150A",
    "HELQ", "PTCD2", "CDX4", "TMLHE", "NPFFR1", "DDO", "ZFYVE19", "CPPED1", "KNDC1", "RBM25",
    "GEMIN4", "MEFV", "UPB1", "NEK4", "TCEAL7", "FUT3", "NDUFAF3", "UNC5C", "IFIT2", "ADH1C",
    "CTSO", "DCTD", "SLC4A3", "MSH5", "MRPL37", "PDK1",
]

#Note: The clinical and RNA Seq files can be txt/tsv/csv/xls/xlsx or other (check pandas read_csv documentation)
separator = '\t'   #If values are separated by other than a tab, change the t for comma or other symbol

#No action needed here. Some files have # to indicate metadata or comments, we read the files excluding these lines
import pandas as pd
clinical_dataset = pd.read_csv(clinical_dataset_directory, sep=separator, comment="#")
RNA_dataset = pd.read_csv(RNA_dataset_directory, sep=separator, comment="#")

#No action needed here, read next comments, click to run this box and check the output
#It is recommended to search if the genes of interest are in the dataset before starting to prevent errors
#If you have genes that were not found, try alternative names and run the box again
#If you still have genes not found, delete them and run the box again, since they are not in the dataset
#Once the -Not found genes- output is empty you can proceed
not_found_genes=[]
for gene in input2:
  if gene not in RNA_dataset.iloc[:, 0].values:
    not_found_genes.append(gene)

    # Delete these genes from the list
    input2.remove(gene)
print("Found genes:", len(input2))
print("These genes were not found and deleted:", len(not_found_genes), "\n", not_found_genes)

Found genes: 920
These genes were not found and deleted: 83 
 ['HBA1', 'ELL', 'PPP4C', 'TUBA8', 'RS1', 'INS', 'NSUN5C', 'PRICKLE4', 'MAK16', 'CRISP3', 'GPR78', 'IPO5', 'TMC4', 'BHMT2', 'TAX1BP3', 'RWDD3', 'PLIN5', 'IL18', 'ZNF705A', 'SDHAF2', 'PLD1', 'DNAJC21', 'GALNT3', 'ACSF2', 'MDS1', 'ZMYM6', 'ZBTB22', 'GUF1', 'CD300C', 'SSTR4', 'DIRC1', 'JUND', 'ZMYND15', 'CSNK1G2', 'CRABP1', 'B3GALT6', 'TNFSF12', 'ZFP82', 'RNF44', 'UMPS', 'C5orf56', 'ZNF696', 'SCO2', 'DNAJC4', 'ZNF175', 'DUX4', 'ZNF620', 'ZKSCAN1', 'ZNF785', 'PCDHA7', 'CFLAR', 'MFSD2', 'RWDD4A', 'LSM8', 'PCYOX1', 'ZNF280B', 'DUX3', 'PI4K2A', 'PCBD2', 'AGBL3', 'MORF4', 'S1PR3', 'KLK2', 'ZIM3', 'ALPK1', 'CD1A', 'ZNF805', 'ZFAND1', 'OR7E5P', 'OSGIN2', 'RPS17P12', 'IRAK2', 'CCDC93', 'SHPK', 'NOTCH4', 'C6orf150', 'NRBP2', 'SSTR3', 'C13orf1', 'LYRM4', 'AQP12A', 'MPO', 'UPB1']


Now you can go to the menu Runtime -> Restart and run all, you will see a moving symbol when the subsections are run and after a few minutes everything will be finished and the results can be seen in the provided output directory in your Google Drive.

If you want, you can open the subsections once finished to see the steps that were made and some additional graphs to preview the distribution of data.

**Note:** The cutoff used for all genes was 0, the data used was Z-scores compared to median of all samples. At the end of section 2 you can see the distribution of Z-scores for each gene and in section 3, step 08 you can change the cutoff for all genes. If you want to change it to any other value, re-run that step (2 code boxes) and see the graphs of the frequency of patients with 0 and 1 (hopefully a somewhat similar number for best results), then re-run the whole notebook (change output folder if you want to compare KM plots with both cutoffs).

## 1-Importing_datasets

**Step 01- Import neccesary packages**

In [3]:
#Google Colab does not include these by default
!pip install XlsxWriter
!pip install lifelines

In [4]:
#These packages are needed to run this code
import pandas as pd                      #Work with dataframes
import numpy as np                       #Do mathematical operations in arrays
import matplotlib.pyplot as plt          #Generate graphs
import math                              #Make some calculations
import xlsxwriter                        #Save dataframes to excel
from lifelines import KaplanMeierFitter  #Generate Kaplan-Meier plots

**Step 02- Check the data was imported correctly**

In [5]:
print("\t The clinical dataset shape is", clinical_dataset.shape, "and it should look like this: ")
clinical_dataset.head()

	 The clinical dataset shape is (2509, 22) and it should look like this: 


,PATIENT_ID,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,...,AGE_AT_DIAGNOSIS,OS_MONTHS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
0,MB-0000,10.0,6.044,NaN,NO,1.0,Positve,NEUTRAL,YES,Post,...,75.65,140.500000,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY
1,MB-0002,0.0,4.020,High,NO,1.0,Positve,NEUTRAL,YES,Pre,...,43.19,84.633333,0:LIVING,LumA,ER+/HER2- High Prolif,Living,Right,YES,Ductal/NST,BREAST CONSERVING
2,MB-0005,1.0,4.030,High,YES,1.0,Positve,NEUTRAL,YES,Pre,...,48.87,163.700000,1:DECEASED,LumB,NaN,Died of Disease,Right,NO,Ductal/NST,MASTECTOMY
3,MB-0006,3.0,4.050,Moderate,YES,1.0,Positve,NEUTRAL,YES,Pre,...,47.68,164.933333,0:LIVING,LumB,NaN,Living,Right,YES,Mixed,MASTECTOMY
4,MB-0008,8.0,6.080,High,YES,1.0,Positve,NEUTRAL,YES,Post,...,76.97,41.366667,1:DECEASED,LumB,ER+/HER2- High Prolif,Died of Disease,Right,YES,Mixed,MASTECTOMY


In [6]:
print("\t The RNA Seq dataset shape is", RNA_dataset.shape, "and it should look like this: ")
RNA_dataset.head()

	 The RNA Seq dataset shape is (24368, 1906) and it should look like this: 


,Hugo_Symbol,Entrez_Gene_Id,MB-0362,MB-0346,MB-0386,MB-0574,MB-0503,MB-0641,MB-0201,MB-0218,...,MB-6122,MB-6192,MB-4820,MB-5527,MB-5167,MB-5465,MB-5453,MB-5471,MB-5127,MB-4313
0,RERE,473.0,-0.7082,1.2179,0.0168,-0.4248,0.4916,0.5156,-1.2105,-0.9309,...,-0.5452,-0.4450,1.8429,1.1092,1.1871,-1.8702,1.1299,0.0481,-0.3357,-1.2562
1,RNF165,494470.0,-0.4419,0.4140,-0.6843,-1.1139,-0.6875,-0.2522,-0.4124,-0.0023,...,0.9537,-1.1564,0.9563,0.0470,-0.2570,3.2290,1.3609,0.6291,0.2281,0.6051
2,CD049690,NaN,0.2236,0.2255,0.5691,0.3545,0.7865,-0.3715,1.9356,-0.1612,...,-0.5783,-0.4329,0.5928,-1.0796,0.1163,-0.0018,0.8035,-0.6178,1.0327,0.8558
3,BC033982,NaN,-2.1485,0.4763,-0.2446,0.2618,-0.2695,-0.8391,-0.6770,0.9853,...,0.1445,-3.1854,-2.2533,1.1311,0.4819,-2.5749,-1.6314,-0.8435,-1.0429,-0.1023
4,PHF7,51533.0,-0.3220,-1.0921,0.2830,-0.2864,0.0772,-0.4976,-0.6453,-0.0506,...,-0.9190,-0.0539,0.7454,0.1631,0.8931,-0.9482,-0.0397,0.5491,-0.0115,4.1846


## 2-Data_filtering

**Step 03- Rearrange the RNA dataframe.**

**Most RNA Seq data is reported having the samples (patients in this case) in columns and genes in rows, but we want to have the same format as the clinical dataframe (patients as rows, to simplify the processing). We also note that, neither the genes, nor the patients are ordered by name/number in the original file, so we correct for that in our dataframe too.**

In [7]:
RNA_dataset = RNA_dataset.sort_values("Hugo_Symbol")               #Sort rows (genes)
RNA_dataset = RNA_dataset.reset_index(drop=True)                   #Reset index, not making other column with new indices
RNA_dataset = RNA_dataset.sort_index(axis=1)                       #Sort columns (patients)
RNA_dataset = RNA_dataset.drop(columns="Entrez_Gene_Id")           #Delete that column
RNA_dataset = RNA_dataset.set_index("Hugo_Symbol").T               #Transpose
RNA_dataset = RNA_dataset.rename_axis("PATIENT_ID").reset_index()  #Patient ID went to be x-axis names

print("\t Now the RNA Seq dataframe shape is", RNA_dataset.shape, "and it looks like this: ")
RNA_dataset.head()
#Ignore Hugo_Symbol, it does not contain that info, just the number of elements in the dataframe

	 Now the RNA Seq dataframe shape is (1904, 24369) and it looks like this: 


Hugo_Symbol,PATIENT_ID,A1BG,A1CF,A2M,A3GALT2,A4GALT,A4GNT,AA001128,AA013027,AA017152,...,hDV103S1,hNp95,hPMS8,hVPS11,mih1/Tx,pKBF-2,psiTPTE22,tMDC,tMDC II,truncated AKR
0,MB-0000,-0.6074,0.5092,0.8489,-2.0901,0.5119,1.2566,-0.6142,-0.3373,-0.9979,...,-0.6644,-2.1755,0.6696,-1.1264,-0.4005,0.7766,-0.4513,-0.4525,0.8682,-1.5789
1,MB-0002,-0.2469,-0.6031,-0.8258,0.9701,0.4970,-0.7856,0.8728,-1.5799,0.6533,...,-0.9280,-0.6968,-1.8526,-1.7330,0.6179,1.5150,-1.2010,0.4439,0.2384,-0.8152
2,MB-0005,0.6546,0.4422,0.9348,1.7684,-0.3699,-1.3290,-0.5119,-0.0041,-0.4030,...,-0.6918,0.1922,1.1901,0.4692,-0.5186,3.5781,1.5827,0.4301,0.4582,-0.5447
3,MB-0006,-2.3896,-0.4547,2.2572,-1.1328,-0.4155,2.7322,0.7113,-0.1766,-2.2133,...,-0.6818,-0.7559,-0.0989,1.1721,0.9832,3.4341,-0.5925,0.8723,0.7092,-0.7848
4,MB-0008,-0.0698,-1.1003,0.8368,-1.3692,-0.7929,-0.9109,-0.3293,-0.3458,0.0146,...,-1.0235,-0.1042,-0.9629,0.2299,3.2432,-0.7779,0.3172,-1.3349,0.6811,0.5632


**Step 04-Select cohort/subset**

**Since RET may play a role mainly in ER+ cancers, we can use the ER IHC column to filter out our dataset (or use a separate file) and remove all the clinical and mRNA data from ER- or with no information about it.**

In [8]:
#Note: There is a typo in the original dataset, it says "Positve" instead of "Positive"
clinical_subset1 = clinical_dataset.query('ER_IHC=="Positve"')

print("Clinical dataset filtered for ER+ patients = subset1")
print("\t Original:", clinical_dataset.shape, "\n \t Subset:", clinical_subset1.shape, "\n \t Removed:", clinical_dataset.shape[0]-clinical_subset1.shape[0])
clinical_subset1.head()

Clinical dataset filtered for ER+ patients = subset1
	 Original: (2509, 22) 
 	 Subset: (1817, 22) 
 	 Removed: 692


,PATIENT_ID,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,...,AGE_AT_DIAGNOSIS,OS_MONTHS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
0,MB-0000,10.0,6.044,NaN,NO,1.0,Positve,NEUTRAL,YES,Post,...,75.65,140.500000,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY
1,MB-0002,0.0,4.020,High,NO,1.0,Positve,NEUTRAL,YES,Pre,...,43.19,84.633333,0:LIVING,LumA,ER+/HER2- High Prolif,Living,Right,YES,Ductal/NST,BREAST CONSERVING
2,MB-0005,1.0,4.030,High,YES,1.0,Positve,NEUTRAL,YES,Pre,...,48.87,163.700000,1:DECEASED,LumB,NaN,Died of Disease,Right,NO,Ductal/NST,MASTECTOMY
3,MB-0006,3.0,4.050,Moderate,YES,1.0,Positve,NEUTRAL,YES,Pre,...,47.68,164.933333,0:LIVING,LumB,NaN,Living,Right,YES,Mixed,MASTECTOMY
4,MB-0008,8.0,6.080,High,YES,1.0,Positve,NEUTRAL,YES,Post,...,76.97,41.366667,1:DECEASED,LumB,ER+/HER2- High Prolif,Died of Disease,Right,YES,Mixed,MASTECTOMY


**Step 05-Select clinical variables to study**

**We have  several variables (columns) in the clinical dataset, we can make easier the dataframe handling and improve visualization if we just pick the variables we want to look at, and make a subset dataframe containing that information.**
**For example, we need the patient IDs plus OS in months at least, and for the KM plot we also need to see how many patients died, so OS Status could be useful, however, Vital Status is better since it indicates the patients who did not die from the disease and thus we should exclude those from our analysis.**

In [9]:
clinical_variables = ["PATIENT_ID", "OS_MONTHS", "VITAL_STATUS"]           #Add/remove column names here
clinical_subset2 = clinical_subset1[clinical_variables]

print("\t Subset2 = ER+ patients + variables of interest", clinical_subset2.shape)
clinical_subset2.head()

	 Subset2 = ER+ patients + variables of interest (1817, 3)


,PATIENT_ID,OS_MONTHS,VITAL_STATUS
0,MB-0000,140.500000,Living
1,MB-0002,84.633333,Living
2,MB-0005,163.700000,Died of Disease
3,MB-0006,164.933333,Living
4,MB-0008,41.366667,Died of Disease


**Step 06- Select genes for RNA Seq filtering**

**At this point we have the clinical information we want to analyze (unprocessed yet). Next, we need to filter the RNA Seq dataset to combine the information of the genes of interest with this dataframe and proceed to process/prepare the data. For this, we enter the genes of interest and filter out all the others (this study has 24,368 in total).**

In [10]:
#We bring back the genes given at the beginning
gene_ref = input1
genes = sorted(input2)

RNA_subset1 = RNA_dataset[["PATIENT_ID", gene_ref] + genes]

print("\t Dataset with genes of interest, shape:", RNA_subset1.shape)
RNA_subset1.head()
#Remember that Hugo_Symbol is nothing other than the index for the rows

	 Dataset with genes of interest, shape: (1904, 922)


Hugo_Symbol,PATIENT_ID,RET,A1BG,AATK,ABHD16B,ABLIM2,ACACB,ACAD8,ACTA2,ACTL8,...,ZNF672,ZNF680,ZNF689,ZNF74,ZNF85,ZNRF2,ZNRF4,ZSCAN10,ZSCAN18,ZXDC
0,MB-0000,-1.1924,-0.6074,0.4330,0.7203,0.7402,3.2632,-0.0676,1.1313,1.0490,...,-1.6492,-0.5563,-0.7072,-1.7857,-1.2386,-0.6136,-0.5906,0.2782,-0.7970,0.1048
1,MB-0002,-0.7346,-0.2469,-0.7811,-0.4436,0.4670,-0.2069,-0.5495,0.5610,0.2229,...,-0.7825,1.4073,1.2040,0.7713,1.8034,-0.1245,-0.6888,0.3939,0.6794,0.6446
2,MB-0005,-0.5291,0.6546,-0.4790,-0.0078,-0.9622,-1.3110,0.5864,1.2911,0.2212,...,-1.7481,2.7934,-0.4441,-1.3336,5.5686,4.0169,1.7027,-0.5116,-0.2900,-2.6086
3,MB-0006,0.9177,-2.3896,1.2821,0.0821,0.8480,-1.0597,-0.7559,1.0042,0.2262,...,-2.7871,1.5456,-1.0595,-1.7254,5.6275,3.2890,0.5035,-0.1731,-1.2113,-1.6190
4,MB-0008,-0.9115,-0.0698,0.2132,0.5185,-2.4299,0.7113,-1.1252,-0.9538,0.1726,...,0.9907,-0.2823,1.1499,0.6938,0.3872,-1.4904,-0.0040,-0.5148,-0.3937,2.9638


**Step 07- Filter ER+ clinical and RNA Seq data**

**For this, we obtained the intersection of the clinical and RNA dataframes through the merge function.**

In [11]:
#There are other ways and parameters (how=)
clinical_RNA_dataset = clinical_subset2.merge(RNA_subset1, how="inner")

print("\t Clinical + RNA Seq dataset for ER+ patients and selected genes", clinical_RNA_dataset.shape)
clinical_RNA_dataset.head()

	 Clinical + RNA Seq dataset for ER+ patients and selected genes (1445, 924)


,PATIENT_ID,OS_MONTHS,VITAL_STATUS,RET,A1BG,AATK,ABHD16B,ABLIM2,ACACB,ACAD8,...,ZNF672,ZNF680,ZNF689,ZNF74,ZNF85,ZNRF2,ZNRF4,ZSCAN10,ZSCAN18,ZXDC
0,MB-0000,140.500000,Living,-1.1924,-0.6074,0.4330,0.7203,0.7402,3.2632,-0.0676,...,-1.6492,-0.5563,-0.7072,-1.7857,-1.2386,-0.6136,-0.5906,0.2782,-0.7970,0.1048
1,MB-0002,84.633333,Living,-0.7346,-0.2469,-0.7811,-0.4436,0.4670,-0.2069,-0.5495,...,-0.7825,1.4073,1.2040,0.7713,1.8034,-0.1245,-0.6888,0.3939,0.6794,0.6446
2,MB-0005,163.700000,Died of Disease,-0.5291,0.6546,-0.4790,-0.0078,-0.9622,-1.3110,0.5864,...,-1.7481,2.7934,-0.4441,-1.3336,5.5686,4.0169,1.7027,-0.5116,-0.2900,-2.6086
3,MB-0006,164.933333,Living,0.9177,-2.3896,1.2821,0.0821,0.8480,-1.0597,-0.7559,...,-2.7871,1.5456,-1.0595,-1.7254,5.6275,3.2890,0.5035,-0.1731,-1.2113,-1.6190
4,MB-0008,41.366667,Died of Disease,-0.9115,-0.0698,0.2132,0.5185,-2.4299,0.7113,-1.1252,...,0.9907,-0.2823,1.1499,0.6938,0.3872,-1.4904,-0.0040,-0.5148,-0.3937,2.9638


**Step 07.1- Inspect dataframes**

**Now we have all the information we neeed together, note that the clinical + RNA dataset has less patients (rows) than the individual datasets. This is because not all ER+ patients were sequenced, thus we keep the intersection between dataframes. **Before starting processing the data to prepare it for KM curve plotting, we can have a quick look at its distribution:**

In [12]:
# This step no longer feasible for this batch, where we are trying to make 900+ plots in a single run
"""
plt.figure(figsize=(5,5))
clinical_RNA_dataset["OS_MONTHS"].plot.hist(title="OS_MONTHS", color="darkgoldenrod", ec="white")
plt.show()

if len(input2)<3:                      #This is just to avoid the error when trying to make a figure of 1 row
    rows=2
else:
    rows = math.ceil((len(input2)+1)/2)

fig, axs = plt.subplots(nrows=rows, ncols=2, figsize = (16,15)) #Adjust nrows to fit more genes
for i, gene in enumerate([gene_ref]+genes):
    if i%2==0 :
        x=int(i/2)
        clinical_RNA_dataset[gene].plot.hist(ax=axs[x, 0], color="black", title=gene, ec="white")
    else:
        x=int(i/2)
        clinical_RNA_dataset[gene].plot.hist(ax=axs[x, 1], color="maroon", title=gene, ec="white")
plt.subplots_adjust(top=0.92, bottom=0.2, left=0.2, right=0.95, hspace=0.95, wspace=0.2)
plt.show()
"""

'\nplt.figure(figsize=(5,5))\nclinical_RNA_dataset["OS_MONTHS"].plot.hist(title="OS_MONTHS", color="darkgoldenrod", ec="white")\nplt.show()\n\nif len(input2)<3:                      #This is just to avoid the error when trying to make a figure of 1 row\n    rows=2\nelse:\n    rows = math.ceil((len(input2)+1)/2)\n        \nfig, axs = plt.subplots(nrows=rows, ncols=2, figsize = (16,15)) #Adjust nrows to fit more genes\nfor i, gene in enumerate([gene_ref]+genes):\n    if i%2==0 :\n        x=int(i/2)\n        clinical_RNA_dataset[gene].plot.hist(ax=axs[x, 0], color="black", title=gene, ec="white")\n    else:\n        x=int(i/2)\n        clinical_RNA_dataset[gene].plot.hist(ax=axs[x, 1], color="maroon", title=gene, ec="white")\nplt.subplots_adjust(top=0.92, bottom=0.2, left=0.2, right=0.95, hspace=0.95, wspace=0.2)\nplt.show()\n'

**From the first graph we can see how many patients have reported overall survival (in months) in the ranges shown. From the other graphs we can see how many patients have mRNA expression within those ranges for each gene, this can be used to stablish cutoffs or thresholds.**

## 3-Data_preparation

**Step 08- Set RNA Seq cutoffs and normalize data**

**Check the distribution and number of datapoints for each condition and decide a cutoff for the genes of interest. Bear in mind that if the cutoff is set too high/low, one condition may have a very small amount of samples labeled with 0 or 1 than others. It is recommended to edit and run the next 2 boxes of code, inspect the graph and decide if the number of samples in each group (low-0, high-1) is adequate for the analysis.**

In [13]:
clinical_RNA_prep1 = clinical_RNA_dataset.copy()
gene_cutoff = 0        #Edit this value to set a custom cutoff (for all genes)

for col in [gene_ref] + genes:
    clinical_RNA_prep1.loc[clinical_RNA_prep1[col].dropna() <= gene_cutoff, col] = 0
    clinical_RNA_prep1.loc[clinical_RNA_prep1[col].dropna() > gene_cutoff, col] = 1
    clinical_RNA_prep1[col] = clinical_RNA_prep1[col].fillna(99)
    clinical_RNA_prep1[col] = clinical_RNA_prep1[col].astype(int)

print("\t Updated dataframe with low=0 and high=1 gene expression", clinical_RNA_prep1.shape)
clinical_RNA_prep1.head()

	 Updated dataframe with low=0 and high=1 gene expression (1445, 924)


,PATIENT_ID,OS_MONTHS,VITAL_STATUS,RET,A1BG,AATK,ABHD16B,ABLIM2,ACACB,ACAD8,...,ZNF672,ZNF680,ZNF689,ZNF74,ZNF85,ZNRF2,ZNRF4,ZSCAN10,ZSCAN18,ZXDC
0,MB-0000,140.500000,Living,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,1,0,1
1,MB-0002,84.633333,Living,0,0,0,0,1,0,0,...,0,1,1,1,1,0,0,1,1,1
2,MB-0005,163.700000,Died of Disease,0,1,0,0,0,0,1,...,0,1,0,0,1,1,1,0,0,0
3,MB-0006,164.933333,Living,1,0,1,1,1,0,0,...,0,1,0,0,1,1,1,0,0,0
4,MB-0008,41.366667,Died of Disease,0,0,1,1,0,1,0,...,1,0,1,1,1,0,0,0,0,1


**Basically, we will have 4 groups based on expression of RET and one more gene of interest at the time, which hopefully will have a similar number of samples (patients) to analyze (see height of bars below):**

**1) Low RET/Low GeneX = (-/-)**

**2) Low RET/High GeneX = (-/+)**

**3) High RET/Low GeneX = (+/-)**

**4) High RET/High GeneX = (+/+)**

In [14]:
# This step no longer feasible for this batch, where we are trying to make 900+ plots in a single run
"""
fig, axs = plt.subplots(nrows=rows, ncols=2, figsize = (16,15)) #Adjust nrows to fit more genes
for i, gene in enumerate([gene_ref]+genes):
    if i%2==0 :
        x=int(i/2)
        clinical_RNA_prep1[gene].plot.hist(ax=axs[x, 0], color="black", bins=3, title=gene, ec="white")
    else:
        x=int(i/2)
        clinical_RNA_prep1[gene].plot.hist(ax=axs[x, 1], color="maroon", bins=3, title=gene, ec="white")
plt.subplots_adjust(top=0.92, bottom=0.2, left=0.2, right=0.95, hspace=0.95, wspace=0.2)
plt.show()
"""

'\nfig, axs = plt.subplots(nrows=rows, ncols=2, figsize = (16,15)) #Adjust nrows to fit more genes\nfor i, gene in enumerate([gene_ref]+genes):\n    if i%2==0 :\n        x=int(i/2)\n        clinical_RNA_prep1[gene].plot.hist(ax=axs[x, 0], color="black", bins=3, title=gene, ec="white")\n    else:\n        x=int(i/2)\n        clinical_RNA_prep1[gene].plot.hist(ax=axs[x, 1], color="maroon", bins=3, title=gene, ec="white")\nplt.subplots_adjust(top=0.92, bottom=0.2, left=0.2, right=0.95, hspace=0.95, wspace=0.2)\nplt.show()\n'

**Step 09-Make Vital status numerical**

**Previously, we inspected the first 5 rows of the dataframe (.head method), however, it is a good practise to check the metadata or inspect the different values a column could have. If we have a quick look at the bottom 5 rows of the dataframe (.tail method), we notice that some patients have  "Died of Other Reasons"  (not due to breast cancer). Since these may cause noise with the OS data, we will filter them out.**

**Note: Seek someone with more expertise in KM curves to determine if these patients could be kept and their OS times used as "Living" or some other strategy. For simplicity and since we have a big dataset, we recommend here to just filter them out.**

In [15]:
print("\t This are the last 5 columns -bottom- of the dataframe: ")
clinical_RNA_prep1.tail()

	 This are the last 5 columns -bottom- of the dataframe: 


,PATIENT_ID,OS_MONTHS,VITAL_STATUS,RET,A1BG,AATK,ABHD16B,ABLIM2,ACACB,ACAD8,...,ZNF672,ZNF680,ZNF689,ZNF74,ZNF85,ZNRF2,ZNRF4,ZSCAN10,ZSCAN18,ZXDC
1440,MB-7295,196.866667,Living,1,1,0,1,1,0,0,...,0,1,1,1,1,1,0,0,0,0
1441,MB-7296,44.733333,Died of Disease,1,0,0,1,0,0,0,...,1,0,0,0,1,1,1,0,0,0
1442,MB-7297,175.966667,Died of Disease,0,1,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0
1443,MB-7298,86.233333,Died of Other Causes,1,1,1,1,1,0,0,...,0,1,1,0,1,1,1,0,0,0
1444,MB-7299,201.900000,Died of Other Causes,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [16]:
#This box shows how we filtered out the patients who died from unrelated reasons
#The next line will keep the text shown, include any other text that may need to be kept
clinical_RNA_prep2 = clinical_RNA_prep1[clinical_RNA_prep1["VITAL_STATUS"].isin(["Living", "Died of Disease"])]
clinical_RNA_prep2 = clinical_RNA_prep2.reset_index(drop=True) #Reset index to prevent confusion

print("\t Keeping just the Living and Died of Disease: ", clinical_RNA_prep2.shape)
print("\t Removed: ", clinical_RNA_prep1.shape[0]-clinical_RNA_prep2.shape[0])
clinical_RNA_prep2.tail()

	 Keeping just the Living and Died of Disease:  (1034, 924)
	 Removed:  411


,PATIENT_ID,OS_MONTHS,VITAL_STATUS,RET,A1BG,AATK,ABHD16B,ABLIM2,ACACB,ACAD8,...,ZNF672,ZNF680,ZNF689,ZNF74,ZNF85,ZNRF2,ZNRF4,ZSCAN10,ZSCAN18,ZXDC
1029,MB-7293,199.233333,Living,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,1,1,0
1030,MB-7294,82.733333,Died of Disease,0,0,1,1,0,0,0,...,0,1,1,0,1,0,1,1,1,0
1031,MB-7295,196.866667,Living,1,1,0,1,1,0,0,...,0,1,1,1,1,1,0,0,0,0
1032,MB-7296,44.733333,Died of Disease,1,0,0,1,0,0,0,...,1,0,0,0,1,1,1,0,0,0
1033,MB-7297,175.966667,Died of Disease,0,1,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,0


**Now, we need to modify the Living and Died of Disease values to make it a numerical classification for KM analysis. We will assign 0 to "Living patients" and 1 to "Died of Disease" patients.**

In [17]:
clinical_RNA_prep3 = clinical_RNA_prep2.copy()

clinical_RNA_prep3["VITAL_STATUS"] = clinical_RNA_prep3["VITAL_STATUS"].apply(lambda x: x.replace("Living", "0"))
clinical_RNA_prep3["VITAL_STATUS"] = clinical_RNA_prep3["VITAL_STATUS"].apply(lambda x: x.replace("Died of Disease", "1"))
clinical_RNA_prep3["VITAL_STATUS"] = pd.to_numeric(clinical_RNA_prep3["VITAL_STATUS"])

print("\t This is our dataframe fully processed and ready for KM plotting:", clinical_RNA_prep3.shape)
clinical_RNA_prep3.head()

	 This is our dataframe fully processed and ready for KM plotting: (1034, 924)


,PATIENT_ID,OS_MONTHS,VITAL_STATUS,RET,A1BG,AATK,ABHD16B,ABLIM2,ACACB,ACAD8,...,ZNF672,ZNF680,ZNF689,ZNF74,ZNF85,ZNRF2,ZNRF4,ZSCAN10,ZSCAN18,ZXDC
0,MB-0000,140.500000,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,1,0,1
1,MB-0002,84.633333,0,0,0,0,0,1,0,0,...,0,1,1,1,1,0,0,1,1,1
2,MB-0005,163.700000,1,0,1,0,0,0,0,1,...,0,1,0,0,1,1,1,0,0,0
3,MB-0006,164.933333,0,1,0,1,1,1,0,0,...,0,1,0,0,1,1,1,0,0,0
4,MB-0008,41.366667,1,0,0,1,1,0,1,0,...,1,0,1,1,1,0,0,0,0,1


## 4-KM_plot

**Step 10- Preparing for KM Fitter**

**For this type of plot there are packages we can use to generate them without having to calculate everything on our own and making our graph from scratch. We imported the one for KM plots (within lifelines) and we will create an object for it to feed with our dataset. First, we bring our clinical+RNA fully-processed dataset.**

In [18]:
dataKM = clinical_RNA_prep3.copy()     #For safety we generate a new variable for the next steps
survival_50 = pd.DataFrame(columns=["Gene", "LL_50%", "LH_50%", "HL_50%", "HH_50%"])  #We will collect the time for 50% survival

**Step 11- Feed the KMF object**

**Our goal is to find the number of days a patient survived before they died. Our event of interest will be “death” which is stored in the “VITAL_STATUS” column. The first argument our KM fitter takes is the timeline for our experiment, stored in the "OS_MONTHS" column.**

In [19]:
def Make_KMplot(gene):

    KMF_LL = KaplanMeierFitter()
    KMF_LH = KaplanMeierFitter()
    KMF_HL = KaplanMeierFitter()
    KMF_HH = KaplanMeierFitter()

    LL = dataKM[(dataKM[gene_ref]==0) & (dataKM[gene]==0)]
    LH = dataKM[(dataKM[gene_ref]==0) & (dataKM[gene]==1)]
    HL = dataKM[(dataKM[gene_ref]==1) & (dataKM[gene]==0)]
    HH = dataKM[(dataKM[gene_ref]==1) & (dataKM[gene]==1)]

    stop = 0.95 #Edit this if you want to plot more/less than 95% of datapoints
    LL_x_stop = int(len(LL.index)*stop)
    LH_x_stop = int(len(LH.index)*stop)
    HL_x_stop = int(len(HL.index)*stop)
    HH_x_stop = int(len(HH.index)*stop)

    print("LL:", LL.shape, "LH:", LH.shape, "HL:", HL.shape, "HH:", HH.shape,)

    KMF_LL.fit(durations =  LL["OS_MONTHS"], event_observed = LL["VITAL_STATUS"], label="Low-Low")
    KMF_LH.fit(durations =  LH["OS_MONTHS"], event_observed = LH["VITAL_STATUS"], label="Low-High")
    KMF_HL.fit(durations =  HL["OS_MONTHS"], event_observed = HL["VITAL_STATUS"], label="High-Low")
    KMF_HH.fit(durations =  HH["OS_MONTHS"], event_observed = HH["VITAL_STATUS"], label="High-High")

    fig = plt.figure(figsize=(12,9))
    KMF_LL.plot(ci_show=True, iloc=slice(0, LL_x_stop))
    KMF_LH.plot(ci_show=True, iloc=slice(0, LH_x_stop))
    KMF_HL.plot(ci_show=True, iloc=slice(0, HL_x_stop))
    KMF_HH.plot(ci_show=True, iloc=slice(0, HH_x_stop))

    plt.title("Kaplan-Meier Estimate For " + gene_ref + " and " + gene + " expression")
    plt.xlabel("Number of months")
    plt.ylabel("Probability of survival")
    plt.savefig(output_directory + "/" + gene_ref + "-" + gene, dpi=600)
    plt.close()

    survival_50.loc[i] = [gene] + [KMF_LL.median_survival_time_] + [KMF_LH.median_survival_time_] + [KMF_HL.median_survival_time_] + [KMF_HH.median_survival_time_]


**Step 12- Run KFM plotting function**

In [20]:
for i,gene in enumerate(genes):
    Make_KMplot(gene)

LL: (246, 924) LH: (251, 924) HL: (236, 924) HH: (301, 924)
LL: (284, 924) LH: (213, 924) HL: (274, 924) HH: (263, 924)
LL: (251, 924) LH: (246, 924) HL: (252, 924) HH: (285, 924)
LL: (281, 924) LH: (216, 924) HL: (283, 924) HH: (254, 924)
LL: (208, 924) LH: (289, 924) HL: (243, 924) HH: (294, 924)
LL: (229, 924) LH: (268, 924) HL: (296, 924) HH: (241, 924)
LL: (176, 924) LH: (321, 924) HL: (240, 924) HH: (297, 924)
LL: (290, 924) LH: (207, 924) HL: (335, 924) HH: (202, 924)
LL: (205, 924) LH: (292, 924) HL: (272, 924) HH: (265, 924)
LL: (254, 924) LH: (243, 924) HL: (258, 924) HH: (279, 924)
LL: (245, 924) LH: (252, 924) HL: (308, 924) HH: (229, 924)
LL: (271, 924) LH: (226, 924) HL: (318, 924) HH: (219, 924)
LL: (235, 924) LH: (262, 924) HL: (319, 924) HH: (218, 924)
LL: (260, 924) LH: (237, 924) HL: (227, 924) HH: (310, 924)
LL: (272, 924) LH: (225, 924) HL: (370, 924) HH: (167, 924)
LL: (263, 924) LH: (234, 924) HL: (285, 924) HH: (252, 924)
LL: (253, 924) LH: (244, 924) HL: (302, 

In [21]:
#We save the original datased and the processed one into an excel file
dataframes_output = output_directory + "/" + output_file_name

writer = pd.ExcelWriter(dataframes_output, engine='xlsxwriter')
clinical_RNA_dataset.to_excel(writer, sheet_name='Original')
clinical_RNA_prep3.to_excel(writer, sheet_name='Processed')
survival_50.to_excel(writer, sheet_name='Survival_50%')
writer.save()

<ipython-input-21-cc1710d1a013>:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


**For more examples of KM plot, Hazard plots, log-Rank test and Cox regression see this source:**

https://medium.com/towards-artificial-intelligence/survival-analysis-with-python-tutorial-how-what-when-and-why-19a5cfb3c312
